<a href="https://colab.research.google.com/github/saraHuang/LLM_study/blob/main/TRL_%E6%95%99%E5%AD%B8_SFTTrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 作者聯絡方式與社群媒體

如果您有任何疑問或想要進一步交流， 也歡迎私訊聯絡我，或隨時關注我的社群媒體：

* **GitHub**： [我的 GitHub 連結](https://github.com/Heng-xiu)  
* **Hugging Face**： [我的 Hugging Face 連結](https://huggingface.co/Heng666)
* **部落格**： [我的 Medium 連結](https://r23456999.medium.com/)

感謝大家的支持，也希望透過這些管道與更多對生成式 AI、Agentic AI System  
或其他技術領域感興趣的朋友們進行討論和交流！

<div class="align-center">
  <a href="https://ko-fi.com/hengshiousheu"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a>
</div>


# 第 1 章：為何需要微調大型語言模型 (LLM)？

大型語言模型 (Large Language Models, LLM) 在近年來展現了驚人的能力，能生成連貫的文字、回答複雜問題、進行翻譯等。然而，即便這些模型擁有強大的通用知識，它們在面對特定任務或領域時，往往還需要進一步的「客製化」才能發揮最佳效能。這就是微調 (Fine-tuning) 的核心目的。

## 1.1 微調 (Fine-tuning) 的角色與類型
微調是指在模型完成大規模預訓練後，使用較小、但與特定任務相關的資料集對模型進行額外訓練的過程。這使得模型能夠將其通用知識應用於更精確的應用場景。

微調主要有兩種常見類型：

1. 全模型微調 (Full Fine-tuning)：
 - 顧名思義，這會更新模型的所有參數。
 - 優勢： 通常能達到最佳的性能表現。
 - 劣勢： 需要大量的計算資源 (GPU 記憶體和算力)，訓練時間長，且會產生與原始模型大小相同的檢查點 (checkpoint)。

2. 參數高效微調 (Parameter-Efficient Fine-Tuning, PEFT)：
 - 這是一種更聰明的微調方式，只更新模型中一小部分參數，或在模型旁新增少量可訓練的「適配器」(adapters)。
 - 優勢： 大幅降低計算資源需求、顯著縮短訓練時間、微調後的模型檔案非常小，方便儲存與部署。
 - 劣勢： 在某些複雜任務上，性能可能略遜於全模型微調 (但差距通常不大，且優勢遠大於劣勢)。

| 本 Notebooks 將主要聚焦在 PEFT 的一種流行技術 LoRA。

## 1.2 監督式微調 (Supervised Fine-tuning, SFT) 簡介

在多種微調策略中，**監督式微調 (Supervised Fine-tuning, SFT) **是最直接且有效的方法之一。

SFT 的核心概念是： 使用包含明確輸入 (Prompt) 和對應理想輸出 (Completion/Response) 的標註資料集來訓練模型。您可以想像這就像給模型看大量的「範例對話」或「問題-答案對」，並告訴它在特定輸入下應該產生什麼樣的輸出。

例如，如果您想訓練一個能回答特定領域問題的模型，您可以提供這樣的資料：
 - 輸入： "台北101的高度是多少公尺？"
 - 輸出： "台北101的高度是508公尺。"

透過大量的這類範例，模型會學習到如何根據輸入指令產生符合預期的回應。SFT 通常是建構特定領域或指令遵循型 LLM 的第一步。

## 1.3 Hugging Face 生態系與 trl 函式庫簡介
在 LLM 的世界裡，Hugging Face 無疑是領頭羊。它提供了一個龐大的生態系，包含：
 - Models Hub： 數十萬個預訓練模型，涵蓋各種任務和語言。
 - Datasets Hub： 大量公開資料集，方便研究與訓練。
 - Transformers 函式庫： 用於載入、使用和訓練各種 Transformer 模型的核心工具。
 - Accelerate 函式庫： 簡化分散式訓練的工具。
 - PEFT 函式庫： 專門用於參數高效微調的工具。

而我們本次主要使用的 **trl (Transformer Reinforcement Learning)** 函式庫，則是 Hugging Face 生態系中的一個重要成員。它提供了一系列用於強化學習 (RL) 和監督式微調 (SFT) LLM 的工具。其中，SFTTrainer 就是一個高度抽象和易用的 API，讓您可以僅用幾行程式碼就能完成複雜的監督式微調任務，大幅降低了實作的門檻。



---



# 第 2 章：環境建置與前置準備
在我們開始微調 LLM 之前，首先要確保我們的開發環境已經準備就緒。一個穩定且配置正確的環境是成功訓練模型的第一步。

> GPU 啟用： 請確保您的 Colab Notebook 已啟用 GPU。點擊菜單欄的 執行階段 (Runtime) -> 變更執行階段類型 (Change runtime type)，然後在 硬體加速器 (Hardware accelerator) 中選擇 GPU。

## 2.1  首先，來登入 HuggingFace

由於我們將從 Hugging Face hub 下載基礎模型 `microsoft/phi-2`，並將我們量化過的模型上傳回 Hugging Face hub，所以讓我們先登入 Hugging Face。

#### Google Colab 新功能
我將我的 Hugging Face token 存儲在左側的秘密標籤中。將我的 token 儲存在這個秘密標籤的好處是，我不會在筆記本中暴露 token，且我可以將這個秘密配置應用於我所有的 Colab 筆記本。

In [ ]:
from google.colab import userdata
from huggingface_hub import HfApi

HF_TOKEN = userdata.get("HF_TOKEN")

api = HfApi(token=HF_TOKEN)
username = api.whoami()['name']
print(username)

Heng666


##2.2 必要 Python 套件安裝
無論您是使用 Conda 環境還是 Google Colab，接下來都需要安裝進行 LLM 微調所需的 Python 套件。請在您啟用的 Conda 環境中 (或 Colab 儲存格中) 執行以下指令：

*   transformers: Hugging Face 模型的基礎，
用於載入模型和分詞器。
*   datasets: Hugging Face 資料集函式庫，用於載入和處理資料。
*   trl: 我們的微調主力，提供 SFTTrainer。
*   peft: 參數高效微調函式庫，用於 LoRA 等技術。
*   accelerate: Hugging Face 的加速工具，有助於訓練優化。
*   bitsandbytes: 用於 8-bit/4-bit 量化，節省記憶體。

In [ ]:
!pip install --quiet transformers datasets trl peft accelerate bitsandbytes

## 2.3 GPU 驅動與 CUDA 支援確認

LLM 的訓練需要大量的計算資源，幾乎必須仰賴 GPU (Graphics Processing Unit)。因此，確認您的環境是否正確偵測到 GPU 並支援 CUDA 是至關重要的一步。

CUDA 是 NVIDIA 提供的平行運算平台和程式設計模型，允許軟體使用 GPU 進行通用計算。PyTorch (一個流行的深度學習框架) 透過 CUDA 來利用 NVIDIA GPU 的運算能力。

請執行以下 Python 程式碼，檢查您的 PyTorch 環境是否已正確偵測到 CUDA：

In [ ]:
# 2.3.1 檢查 PyTorch 是否偵測到 CUDA
import torch

print(f"PyTorch 是否支援 CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"目前使用的 CUDA 裝置名稱: {torch.cuda.get_device_name(0)}")
    print(f"CUDA 裝置數量: {torch.cuda.device_count()}")
    # 可以進一步檢查 CUDA 版本
    print(f"PyTorch 編譯的 CUDA 版本: {torch.version.cuda}")
    # 執行 nvidia-smi (僅限 Linux/Windows 終端機，Colab 可直接執行)
    # !nvidia-smi
else:
    print("警告：未偵測到 CUDA。模型訓練將在 CPU 上運行，速度會非常慢。")
    print("請檢查您的 GPU 驅動程式安裝、CUDA Toolkit 設定以及 PyTorch 的 CUDA 支援。")

PyTorch 是否支援 CUDA: True
目前使用的 CUDA 裝置名稱: Tesla T4
CUDA 裝置數量: 1
PyTorch 編譯的 CUDA 版本: 12.4


In [ ]:
!nvidia-smi

Tue Jul  1 03:11:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

> 【重要提醒】：如果 torch.cuda.is_available() 回傳 False，您可能需要：

確認您的電腦有 NVIDIA GPU。

安裝正確版本的 NVIDIA 顯示卡驅動程式。

安裝與您 PyTorch 版本相容的 CUDA Toolkit。

對於 Colab 用戶，請再次確認您已在執行階段中選擇了 GPU。



---



# 第 3 章：SFTTrainer 核心概念與基礎應用
本章將深入介紹 `SFTTrainer` 的核心概念，並帶您完成第一個基礎的監督式微調實作。我們將以經典的 `facebook/opt-350m` 模型為基礎，並使用 imdb (電影評論情感分析) 資料集作為訓練範例。






## 3.1 `SFTTrainer` 是什麼？為什麼使用它？
SFTTrainer 是 Hugging Face trl 函式庫提供的一個高階 API，專為監督式微調 (Supervised Fine-tuning, SFT) 大型語言模型而設計。

它將許多微調過程中的複雜步驟抽象化，讓您可以僅用幾行程式碼就完成原本繁瑣的工作，例如：

 - 資料預處理： 自動處理資料集的 Tokenization (分詞)、填充 (padding) 和截斷 (truncation)。

 - 模型載入： 輕鬆載入預訓練模型。

 - 訓練迴圈管理： 自動處理訓練批次、梯度計算、模型更新、日誌記錄和檢查點儲存。

 - 內建優化： 支援多種優化技術，如 PEFT (LoRA)、資料打包等。

使用 SFTTrainer，您可以更專注於模型、資料和超參數的選擇，而不是陷入繁瑣的訓練流程實作細節。

## 3.2 程式碼實作：使用 imdb 資料集進行基礎 SFT
本節將帶您實際操作，使用 `facebook/opt-350m` 模型和 `stanfordnlp/imdb` 資料集來完成您的第一個基礎 SFT 任務。imdb 資料集包含了大量的電影評論文本，我們將用它來微調模型，使其能更好地理解和生成類似評論的文本。

### 3.2.1 資料集載入與探索

首先，我們從 Hugging Face Datasets Hub 載入 imdb 資料集的訓練部分。



In [ ]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

print("載入 'stanfordnlp/imdb' 訓練資料集...")
# `split="train"` 指定我們只載入訓練集
dataset = load_dataset("stanfordnlp/imdb", split="train")
print("資料集載入完成！")

# 顯示資料集的結構和一些基本資訊
print("\n資料集結構:")
print(dataset)

# 顯示資料集的第一筆範例，了解其內容
print("\n資料集第一筆範例:")
print(dataset[0])
print(f"第一筆範例的文本內容 (欄位 'text'):\n{dataset[0]['text']}")
print(f"第一筆範例的標籤 (欄位 'label'): {dataset[0]['label']} (0 代表負面，1 代表正面)")
print(f"資料集總大小: {len(dataset)} 筆資料")

載入 'stanfordnlp/imdb' 訓練資料集...


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

資料集載入完成！

資料集結構:
Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

資料集第一筆範例:
{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 yea

從輸出中，您可以看到 imdb 資料集包含 text (電影評論文本) 和 label (情感標籤) 兩個欄位。在 SFT 中，SFTTrainer 會自動使用 text 欄位作為模型的主要輸入進行語言建模任務。

### 3.2.2 模型與 Tokenizer 初始化
接著，我們載入用於微調的基礎模型 facebook/opt-350m 及其對應的分詞器 (Tokenizer)。分詞器負責將原始文本轉換為模型能處理的數字 ID 序列。

In [ ]:
# 3.3.2.1 載入預訓練模型與分詞器
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch # 導入 torch 以指定數據類型

model_id = "facebook/opt-350m"
print(f"載入模型: {model_id}...")

# AutoModelForCausalLM 適用於因果語言模型 (如 GPT 系列，用於文本生成)
# 將模型載入到 GPU 上 (如果可用) 並使用 bfloat16 精度 (如果 GPU 支援)，以節省記憶體並加速
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto" # 自動將模型層分佈到可用的裝置上 (CPU 或 GPU)
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 確保模型的 `pad_token_id` 與分詞器的 `pad_token_id` 或 `eos_token_id` 一致。
# 這對於批次處理長度不一的序列非常重要，確保填充 (padding) 正確處理。
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id

print("模型與分詞器載入完成！")

載入模型: facebook/opt-350m...


config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

模型與分詞器載入完成！


> 【實戰知識】

- `AutoTokenizer`： 同樣是通用類，會自動載入模型 ID 對應的分詞器。
- `torch_dtype`： 設定模型參數的資料類型。torch.bfloat16 (Brain Floating Point) 是一種半精度浮點數格式，能在保持良好精度的同時顯著降低記憶體佔用並加速訓練，推薦在支援的 GPU 上使用。torch.float16 (Half Precision) 也是常用選項。
- `device_map="auto"`： 告訴 Hugging Face Transformers 自動判斷如何將模型的分層分佈到可用的 GPU 或 CPU 上，這在多 GPU 環境或記憶體有限時非常有用。

### 3.2.3 `SFTConfig` 關鍵參數解析

`SFTConfig` 類別 是 `SFTTrainer` 的核心配置，您可以在這裡定義所有訓練相關的超參數。理解這些參數的用途對您的訓練過程至關重要。

In [ ]:
# 3.3.3.1 設定 SFTTrainer 的訓練參數
from trl import SFTConfig

print("開始設定 SFTConfig 訓練參數...")
training_args = SFTConfig(
    # == 模型輸出與保存相關 ==
    output_dir="/tmp/sft_imdb_model", # 模型訓練結果和檢查點的輸出目錄。
                                      # 【建議】可以設為您希望儲存模型的路徑。
    logging_steps=10, # 每隔多少步記錄一次訓練日誌 (例如損失值、學習率)。
    save_steps=100, # 每隔多少步儲存一次模型檢查點 (checkpoint)。
    save_total_limit=2, # 限制檢查點的數量，只保留最新的 N 個。
    report_to="none", # 不使用任何報告工具 (如 Weights & Biases)，如果需要集成可以設為 "wandb" 等。

    # == 訓練過程相關 ==
    num_train_epochs=1, # 訓練的總 epoch 數。一個 epoch 代表模型看過整個訓練資料集一次。
                       # 【動手試試看】對於初次嘗試，1 個 epoch 足夠。您可以嘗試 2-3 個 epoch，觀察損失下降情況。
    max_length=512, # 輸入序列的最大長度。較長的文本會被截斷，較短的會被填充。
                    # 【重要】此參數直接影響記憶體使用量和訓練速度。
                    # 預設值是 min(tokenizer.model_max_length, 1024)。對於 IMDB 評論，512 通常足夠。
                    # 【動手試試看】您可以嘗試將此值調整為 256 或 1024，觀察對訓練時間和記憶體使用的影響。

    # == 優化器和學習率排程 ==
    learning_rate=2e-4, # 訓練的初始學習率。這是非常重要的超參數。
                       # 【動手試試看】常見範圍是 1e-5 到 5e-5。太高可能導致不穩定，太低可能收斂慢。
    per_device_train_batch_size=4, # 每個 GPU (或 CPU) 裝置的訓練批次大小。
                                  # 【重要】此參數直接影響記憶體使用量。記憶體足夠可以設更大。
    gradient_accumulation_steps=2, # 梯度累積步數。這允許您在不增加實際批次大小的情況下，模擬更大的批次。
                                  # 例如，如果 per_device_train_batch_size=4 且 gradient_accumulation_steps=2，
                                  # 則有效批次大小為 4 * 2 = 8。有助於在記憶體有限情況下模擬大批次。

    # == 其他優化 ==
    # packing=True, # 啟用資料打包，將多個短序列合併為一個長序列，減少填充，提高 GPU 利用率。
                   # 適用於資料集中包含大量短文本的情況。我們會在第 4 章詳細介紹。
    # peft_config=peft_config, # 如果使用 PEFT (如 LoRA)，在這裡傳入 LoraConfig 物件。我們會在第 5 章詳細介紹。

    # 更多參數請參考 SFTConfig 的官方文件或 Hugging Face TrainingArguments 文件。
)

print("SFTConfig 參數設定完成！")

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.


開始設定 SFTConfig 訓練參數...
SFTConfig 參數設定完成！


### 3.2.4 `SFTTrainer` 初始化與訓練
現在，我們將使用前面準備好的模型、資料集和訓練參數來初始化 SFTTrainer，並開始模型的微調過程。

#### 3.2.4.1 初始化 SFTTrainer

In [ ]:
from trl import SFTTrainer, SFTConfig

print("初始化 SFTTrainer...")
trainer = SFTTrainer(
    model=model, # 傳入我們載入的預訓練模型
    train_dataset=dataset, # 傳入訓練資料集 (IMDB)
    args=training_args, # 傳入 SFTConfig 中設定的訓練參數
    # dataset_text_field="text", # 指定資料集中哪一個欄位包含我們想要模型學習的文本內容。
                               # 對於 IMDB 資料集，文本內容在 'text' 欄位。
)
print("SFTTrainer 初始化完成！")

初始化 SFTTrainer...


Adding EOS to train dataset:   0%|          | 0/25000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/25000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/25000 [00:00<?, ? examples/s]

SFTTrainer 初始化完成！


#### 3.2.4.2 啟動訓練

In [ ]:
print("開始訓練模型...")
trainer.train()
print("模型訓練完成！")

開始訓練模型...


Step,Training Loss
10,4.193900
20,3.899100
30,3.887100


KeyboardInterrupt: 

In [ ]:
print("清理環境...")
torch.cuda.empty_cache()

清理環境...


#### 3.2.4.3 儲存訓練後的模型

In [ ]:
# 訓練完成後，將模型儲存到 output_dir 指定的路徑。
# 這將保存模型的權重、分詞器配置以及訓練配置，方便後續載入和使用。
output_path = training_args.output_dir # 直接使用 SFTConfig 中定義的輸出目錄
trainer.save_model(output_path)
print(f"訓練完成的模型已儲存至: {output_path}")

#### 3.2.4.4 如何載入微調後的模型進行推論 (範例程式碼)

In [ ]:
from transformers import pipeline
loaded_model = AutoModelForCausalLM.from_pretrained(output_path)
loaded_tokenizer = AutoTokenizer.from_pretrained(output_path)
generator = pipeline("text-generation", model=loaded_model, tokenizer=loaded_tokenizer, device=0) # device=0 指第一個 GPU
result = generator("This movie was", max_new_tokens=50, num_return_sequences=1)
print("\n微調後模型推論範例:")
print(result[0]['generated_text'])



---



# 第 4 章：資料格式化與 Chat Template 進階處理
在 LLM 微調中，資料的格式化方式對模型的學習效果至關重要。本章將探討 `SFTTrainer` 支援的資料格式，並深入介紹如何處理不同格式的資料集，特別是針對對話模型極為重要的 Chat Template，讓模型能理解對話的結構。

## 4.1 SFTTrainer 支援的資料格式
`SFTTrainer` 設計得相當靈活，可以直接處理幾種常見的資料格式，無需您進行複雜的預處理。這是因為 `SFTTrainer` 在內部會自動呼叫分詞器的 `apply_chat_template` 方法來將這些結構化的資料轉換為模型可訓練的文本格式。

### 4.1.1 對話式 (Conversational) 格式
這種格式常用於訓練聊天機器人，其中對話內容由一系列訊息組成，每條訊息都有 `role` (角色，如 `system`, `user`, `assistant`) 和 content (內容)。

範例結構

```python
{
  "messages": [
    {"role": "system", "content": "您是一位樂於助人的 AI 助理。"},
    {"role": "user", "content": "請問法國的首都是哪裡？"},
    {"role": "assistant", "content": "法國的首都是巴黎。"}
  ]
}
```

### 4.1.2 指令式 (Instruction) 格式
這種格式常用於訓練模型遵循特定指令並生成回應，包含 prompt (指令/問題) 和 completion (理想的生成文本/答案)。

範例結構
```python
{
  "prompt": "請給我一個關於貓的短故事。",
  "completion": "在一條古老的巷子裡，住著一隻名叫咪咪的橘貓。牠最喜歡在陽光下打盹，並追逐路過的蝴蝶..."
}
```

說明：
如果您的資料集是上述兩種格式之一，您可以直接將其傳給 SFTTrainer。SFTTrainer 會根據模型的 tokenizer.apply_chat_template 方法自動將 messages 或 prompt 和 completion 欄位轉換為模型可訓練的文本序列。

例如，對於對話式格式，SFTTrainer 會將上述 JSON 範例自動轉換為類似 "[INST] 法國的首都是哪裡？ [/INST] 法國的首都是巴黎。" 這樣的文本，具體格式取決於模型分詞器的 Chat Template。

## 4.2 自訂資料格式化函式 (formatting_prompts_func)
並非所有資料集都直接符合 `SFTTrainer` 支援的標準格式。當您的資料集具有自訂的欄位結構時，您可以提供一個**自訂的格式化函式 (formatting_prompts_func)** 來告訴 SFTTrainer 如何將您的資料轉換成模型可訓練的單一文本字串。

這個函式會接收資料集中的單一範例 (字典)，然後返回一個包含處理後文本的字串。

### 4.2.1 案例情境與程式碼範例
假設您的資料集有 question 和 answer 兩個欄位，而您希望將其格式化為類似 Alpaca 數據集那樣的指令-回應結構：
```
### 指令: {question}
### 回應: {answer}
```
以下是實作方式：

#### 4.2.1.1 - 假設有一個自訂格式的資料集

In [ ]:
# 為了示範，我們手動建立一個簡單的 Dataset 物件
from datasets import Dataset

custom_dataset_data = {
    "question": ["請定義人工智慧。", "地球是什麼形狀？", "說一個笑話。"],
    "answer": ["人工智慧 (AI) 是模擬人類智慧的技術，讓機器能學習、解決問題。", "地球是一個橢球體，接近於球形。", "為什麼科學家不相信原子？因為它們構成了一切！"]
}
custom_dataset = Dataset.from_dict(custom_dataset_data)

print("自訂資料集範例:")
print(custom_dataset[0])

自訂資料集範例:
{'question': '請定義人工智慧。', 'answer': '人工智慧 (AI) 是模擬人類智慧的技術，讓機器能學習、解決問題。'}


#### 4.2.1.2 定義自訂格式化函式


In [ ]:
def formatting_prompts_func(example):
    # 這個函式接收一個資料集中的 example (字典)
    # 它必須返回一個字串，這個字串就是模型訓練時的輸入文本
    text = f"### 指令: {example['question']}\n### 回應: {example['answer']}"
    return text

print("\n格式化函式範例輸出 (第一筆資料):")
print(formatting_prompts_func(custom_dataset[0]))


格式化函式範例輸出 (第一筆資料):
### 指令: 請定義人工智慧。
### 回應: 人工智慧 (AI) 是模擬人類智慧的技術，讓機器能學習、解決問題。


#### 4.2.1.3 將自訂格式化函式傳入 SFTTrainer
我們需要重新初始化模型和分詞器，以確保環境乾淨

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTConfig, SFTTrainer
import torch

model_id_custom = "facebook/opt-350m"
model_custom = AutoModelForCausalLM.from_pretrained(model_id_custom, device_map="auto", torch_dtype=torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16)
tokenizer_custom = AutoTokenizer.from_pretrained(model_id_custom)
if tokenizer_custom.pad_token is None:
    tokenizer_custom.pad_token = tokenizer_custom.eos_token
if model_custom.config.pad_token_id is None:
    model_custom.config.pad_token_id = model_custom.config.eos_token_id

training_args_custom = SFTConfig(
    output_dir="/tmp/custom_formatted_sft_model",
    num_train_epochs=1,
    max_length=512,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    logging_steps=10,
)

print("\n使用自訂格式化函式初始化 SFTTrainer...")
trainer_custom = SFTTrainer(
    model=model_custom,
    train_dataset=custom_dataset, # 使用自訂資料集
    args=training_args_custom,
    formatting_func=formatting_prompts_func, # 傳入我們定義的格式化函式
    # 當使用 formatting_func 時，通常不需要指定 dataset_text_field，
    # 因為數據如何生成文本的邏輯已經在 formatting_func 中了。
)
print("SFTTrainer 準備就緒，")


使用自訂格式化函式初始化 SFTTrainer...


Applying formatting function to train dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

SFTTrainer 準備就緒，




---



## 4.3 Chat Template 深度解析與應用

Chat Template (聊天模板) 是訓練對話型 LLM 時一個極其重要的概念。它定義了模型如何將結構化的對話訊息（如用戶發言、助手回應、系統指令）轉換成模型能夠理解和生成文本的單一字串。這涉及到使用特殊的 Token (符號) 來標記對話的不同部分。

### 4.3.1 為什麼需要 Chat Template？特殊 Token 的重要性

想像一下人類對話，我們會用語氣、停頓、眼神交流來區分誰在說話以及何時結束。對於 LLM 而言，特殊 Token 就是它們理解這些「語氣」和「停頓」的方式。

 - 區分角色： 特殊 Token (例如 `<|im_start|>user`, `<|im_end|>`, `[INST]`, `[/INST]`) 能夠清晰地標示出對話中不同參與者的發言邊界和角色。這讓模型知道哪部分是使用者的輸入、哪部分是助手的回應，以及系統指令。

 - 結構化輸入： 確保模型在訓練和推論時，輸入的對話格式是一致的。如果輸入格式不一致，模型將難以學習正確的回應模式。

 - 控制生成： 某些特殊 Token (如 eos_token，End-of-Sequence Token) 告訴模型何時停止生成。在對話中，這意味著助手知道何時結束自己的發言。

如果沒有正確的 Chat Template，模型可能會：
 - 無法區分對話中的不同角色，導致語氣混亂。
 - 生成不連貫或不符合上下文的回應。
 - 無法在適當的時機停止生成，導致無止盡的重複或亂碼。

### 4.3.2 clone_chat_template() 的作用與使用時機

trl 函式庫提供了 clone_chat_template() 函式，用於為模型和分詞器設定或調整 Chat Template。

1. 主要作用：

 - 新增特殊 Token： 如果原始分詞器缺少 Chat Template 所需的特殊 Token (例如 <|im_start|>, <|im_end|>)，這個函式會將它們添加到分詞器中。

 - 調整詞嵌入層： 由於添加了新的 Token，模型的詞嵌入層 (model.get_input_embeddings()) 的大小需要調整，以容納這些新 Token。clone_chat_template() 會自動處理這個步驟。
 - 設定 chat_template： 為分詞器設定其 chat_template 屬性，這是一個 JINJA 模板字串，定義了如何將 messages 列表轉換為單一的聊天字符串。

2. 使用時機：
 - 當您使用的基礎模型沒有預設的 Chat Template，或者您希望使用不同於模型預設的對話格式時。例如，您想將一個通用模型訓練成遵循 ChatML (OpenAI 風格) 或 Llama-2 的對話格式。
 - 您希望為模型添加新的特殊行為 Token。

3. 特殊情況與重要提示：
 - 部分基礎模型已預設 Chat Template： 許多較新的模型 (例如 Qwen 系列、Llama-2-Chat 等) 在其發布時，其分詞器就已經內建了預設的 Chat Template，並且這些模板可能包含特殊的 Token 和邏輯。在這些情況下，您通常不需要使用 clone_chat_template()，因為分詞器已經知道如何處理對話格式。
 - `eos_token` 的重要性： 對於這些已預設模板的模型，關鍵在於要確保在 SFTConfig 中，`eos_token` (End-of-Sequence Token) 的設定與其 Chat Template 中的結束符號一致。例如，Qwen 模型通常使用 `<|im_end|>` 作為其對話輪次的結束符號和整個對話的結束符號。因此，您可能需要在 SFTConfig 或模型載入時明確設定 eos_token="<|im_end|>"，以確保模型在生成結束時正確停止。



---



# 第 5 章：高效能微調策略與技術

本章將著重介紹**參數高效微調 (Parameter-Efficient Fine-Tuning, PEFT)** 中的 **LoRA (Low-Rank Adaptation)** 技術。這是一種能夠在大幅減少訓練資源消耗的同時，保持甚至提升模型性能的有效方法。學習 LoRA 對於在資源有限的情況下進行 LLM 微調至關重要。

## 5.1 參數高效微調 (Parameter-Efficient Fine-Tuning, PEFT) 與 LoRA

### 5.1.1 PEFT 原理與優勢
傳統的**全模型微調 (Full Fine-tuning)** 雖然效果最好，但需要巨大的計算資源 (特別是 GPU 記憶體和算力)。這對於沒有大量 GPU 集群的個人或小型團隊來說是個巨大的障礙。

**參數高效微調 (PEFT) **技術應運而生，它解決了這個問題。PEFT 的核心思想是：
- **凍結大部分預訓練模型的參數**： 只讓一小部分參數參與訓練。
- **引入少量可訓練的參數或「適配器」(Adapters)**： 這些新增的參數專門用於適應下游任務。

### 5.1.2 LoRA (Low-Rank Adaptation) 參數解析

LoRA 是 PEFT 家族中最流行和有效的技術之一。它的原理是：對於模型中的大型權重矩陣 $W_0$，LoRA 凍結 $W_0$，並在訓練期間通過學習兩個較小的低秩矩陣 A 和 B 來表示權重的更新
$DeltaW=BA$。這樣，$W_0$ 就被 $W_0+BA$ 所取代。

這個巧妙的設計使得我們只需要訓練 $A$ 和 $B$ 這兩個小矩陣的參數，而不是整個 $W_0$，從而大幅減少了可訓練參數的數量。

在 peft 函式庫中，您透過 LoraConfig 類別來設定 LoRA 的行為。以下是一些關鍵參數：

- r (rank)： 這是 LoRA 矩陣 A 和 B 的秩 (rank)。它決定了 LoRA 適配器的大小。

  - 【用途】 較大的 r 值意味著更多的可訓練參數，理論上模型有更大的表達能力，可能帶來更好的性能，但也需要更多的記憶體和計算。

  - 【動手試試看】 常用值是 8, 16, 32, 64。您可以嘗試不同的 r 值，觀察性能和資源消耗的權衡。

- lora_alpha： LoRA 的縮放因子。它通常與 r 成比例。

  - 【用途】 用於調整 LoRA 更新對原始權重的影響強度。較大的 lora_alpha 意味著 LoRA 的影響更顯著。

- lora_dropout： LoRA 層的 Dropout 機率。

  - 【用途】 在訓練期間隨機丟棄一些 LoRA 參數，有助於防止過擬合。

- target_modules： 一個列表，指定要應用 LoRA 的模塊名稱。

  - 【用途】 通常會選擇模型中大型的線性層，如 Transformer 注意力機制中的查詢 (query, q_proj)、鍵 (key, k_proj)、值 (value, v_proj)、輸出 (output, o_proj) 投影層，以及前饋網絡中的層。

  - 【如何找到名稱】 您可以透過印出模型結構來找到這些層的名稱，例如 print(model)。

- modules_to_save： 一個列表，指定除了 LoRA 適配器之外，還需要進行全量微調並儲存的模塊名稱。

  - 【重要用途】 這對於處理 Chat Template 中新增的特殊 Token (例如 <|im_start|>, <|im_end|>) 至關重要。這些特殊 Token 通常會影響模型的詞嵌入層 (model.get_input_embeddings()) 和最終語言模型頭 (model.get_output_embeddings() 或 lm_head)。

  - 【原因】 如果這些與特殊 Token 相關的層沒有被微調，模型將無法正確理解或生成這些 Token，導致訓練效果不佳或推論時產生無意義的結果。因此，當使用 Chat Template 或添加了新 Token 時，務必將 lm_head 和模型的嵌入層 (通常稱為 embed_tokens 或 wte) 加入 modules_to_save。

- task_type： 任務類型。

  - 【用途】 告訴 PEFT 您正在處理什麼類型的任務，以便它能進行相關的最佳化。對於 LLM 微調，通常是 TaskType.CAUSAL_LM (因果語言模型)。

### 5.1.3 程式碼實作：使用 LoRA 進行微調
現在，我們將示範如何結合 `LoraConfig` 與 `SFTTrainer` 進行參數高效微調。我們將使用 `Qwen/Qwen2-0.5B `作為基礎模型，它是一個預設帶有 `Chat Template` 的模型，這能幫助我們理解 `modules_to_save` 的重要性。



#### 5.1.3.1 載入資料集

In [ ]:
from datasets import load_dataset
# 我們使用 Hugging Face Hub 上的一個對話風格資料集作為範例：trl-lib/Capybara。
# 也可以繼續使用 IMDB 資料集，但對於對話模型來說，Capybara 更具代表性。
print("載入 'trl-lib/Capybara' 訓練資料集...")
dataset_lora = load_dataset("trl-lib/Capybara", split="train")
print("Capybara 資料集載入完成！")
print("\nCapybara 資料集範例 (包含 'messages' 欄位):")
print(dataset_lora[0]["messages"]) # 這是對話式格式

載入 'trl-lib/Capybara' 訓練資料集...
Capybara 資料集載入完成！

Capybara 資料集範例 (包含 'messages' 欄位):
[{'content': 'Recommend a movie to watch.\n', 'role': 'user'}, {'content': 'I would recommend the movie, "The Shawshank Redemption" which is a classic drama film starring Tim Robbins and Morgan Freeman. This film tells a powerful story about hope and resilience, as it follows the story of a young man who is wrongfully convicted of murder and sent to prison. Amidst the harsh realities of prison life, the protagonist forms a bond with a fellow inmate, and together they navigate the challenges of incarceration, while holding on to the hope of eventual freedom. This timeless movie is a must-watch for its moving performances, uplifting message, and unforgettable storytelling.', 'role': 'assistant'}, {'content': 'Describe the character development of Tim Robbins\' character in "The Shawshank Redemption".', 'role': 'user'}, {'content': 'In "The Shawshank Redemption", Tim Robbins plays the character of Andy Du

#### 5.1.3.2 載入基礎模型與分詞器 (這裡使用 Qwen/Qwen2-0.5B 作為範例)


In [ ]:
##Relesase CUDA
print("清理環境...")
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id_lora = "Qwen/Qwen2-0.5B"
print(f"\n載入基礎模型進行 LoRA 微調: {model_id_lora}...")

# 載入模型，同樣使用自動裝置映射和 bfloat16 精度
model_lora = AutoModelForCausalLM.from_pretrained(
    model_id_lora,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto"
)
tokenizer_lora = AutoTokenizer.from_pretrained(model_id_lora)

# 確保模型的 `pad_token_id` 與分詞器的 `pad_token_id` 或 `eos_token_id` 一致。
# 對於 Qwen2，它的 Chat Template 通常以 <|im_end|> 作為 EOS Token。
if tokenizer_lora.pad_token is None:
    tokenizer_lora.pad_token = tokenizer_lora.eos_token
if model_lora.config.pad_token_id is None:
    model_lora.config.pad_token_id = model_lora.config.eos_token_id

print("模型與分詞器載入完成！")


載入基礎模型進行 LoRA 微調: Qwen/Qwen2-0.5B...
模型與分詞器載入完成！


#### 5.1.3.3 設定 LoRA 配置

In [ ]:
model_lora

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [ ]:
from peft import LoraConfig, TaskType

print("\n設定 LoRA 配置 (LoraConfig)...")
peft_config = LoraConfig(
    r=16, # LoRA 的秩，【動手試試看】可以嘗試更大的值如 32, 64，看是否影響性能。
    lora_alpha=32, # LoRA 縮放因子，通常與 r 成比例。
    lora_dropout=0.05, # Dropout 率，用於防止過擬合。
    # 指定 LoRA 應用於哪些模塊。這些是 Qwen2 模型中常見的注意力層投影模塊。
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    # 針對使用 Chat Template 的模型：
    # 如果您的模型使用了特殊 Token (透過 clone_chat_template 添加，或模型自帶)，
    # 並且這些 Token 對應的詞嵌入層和最終的語言模型頭需要被微調，
    # 則必須將 "lm_head" 和 "embed_tokens" (或其他實際的嵌入層名稱) 加入 modules_to_save。
    # 對於 Qwen2-0.5B，它的詞嵌入層通常稱為 `model.embed_tokens`，語言模型頭是 `lm_head`。
    modules_to_save=["lm_head", "embed_tokens"],
    task_type=TaskType.CAUSAL_LM, # 任務類型：因果語言模型。
)
print("LoRA 配置完成！")


設定 LoRA 配置 (LoraConfig)...
LoRA 配置完成！


#### 5.1.3.4 設定 SFTConfig (搭配 LoRA)

In [ ]:
from trl import SFTConfig

training_args_lora = SFTConfig(
    output_dir="./qwen_lora_sft_model", # 儲存 LoRA 適配器和訓練日誌的目錄
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    logging_steps=10,
    save_steps=100,
    learning_rate=2e-4,
    # 對於 Qwen 類模型，確保 EOS Token 與其 Chat Template 的結束符號匹配。
    # Qwen2 通常使用 tokenizer.eos_token 作為 <|im_end|>。
    eos_token=tokenizer_lora.eos_token,
    # 對於對話格式的資料集 (如 Capybara)，確保 `remove_unused_columns` 為 False，
    # 這樣 'messages' 欄位才不會被移除，因為它會被 tokenizer.apply_chat_template 使用。
    remove_unused_columns=False,
    report_to="none",
)

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.


#### 5.1.3.5 初始化 SFTTrainer 與訓練 (結合 LoRA)

In [ ]:
import torch
import gc # 導入 Python 的垃圾回收模組

# 1. 刪除模型和訓練器物件


# 2. 清理 PyTorch 的快取記憶體
# 這會釋放 PyTorch 已經分配但在目前未使用中的 GPU 記憶體。
torch.cuda.empty_cache()

# 3. 執行 Python 的垃圾回收
# 確保所有被刪除的物件都從記憶體中清除。
gc.collect()

print("GPU 記憶體已嘗試釋放。")

GPU 記憶體已嘗試釋放。


In [ ]:
from trl import SFTTrainer

print("\n初始化 SFTTrainer (結合 LoRA)...")
trainer_lora = SFTTrainer(
    model=model_lora, # 傳入基礎模型
    train_dataset=dataset_lora, # 傳入訓練資料集 (Capybara)
    args=training_args_lora, # 傳入 SFTConfig 訓練參數
    peft_config=peft_config, # 【關鍵】傳入 LoRA 配置
    # dataset_text_field="messages", # 對於對話式資料集 (如 Capybara)，指定包含對話的 'messages' 欄位
)
print("SFTTrainer (LoRA) 初始化完成！")


print("\n開始訓練 LoRA 模型...")
trainer_lora.train()
print("LoRA 模型訓練完成！")


初始化 SFTTrainer (結合 LoRA)...


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


SFTTrainer (LoRA) 初始化完成！

開始訓練 LoRA 模型...


Step,Training Loss
10,1.733100
20,1.773000
30,1.800200
40,1.718700
50,1.533700
60,1.766400
70,1.865400
80,1.817300
90,1.842800


KeyboardInterrupt: 

#### 5.1.3.6 儲存 LoRA 適配器

In [ ]:
# 訓練完成後，我們只需要儲存 LoRA 適配器的權重，它們非常小。
# LoRA 適配器可以與原始模型合併後進行推論，或者直接載入在原始模型之上。
lora_output_path = training_args_lora.output_dir # 使用 SFTConfig 中定義的輸出目錄
# trainer_lora.save_model(lora_output_path)
# print(f"訓練後的 LoRA 適配器已儲存至: {lora_output_path}")

#### 5.1.3.7 如何載入 LoRA 適配器並與基礎模型合併以進行推論

In [ ]:
from peft import PeftModel
print("\n--- 載入 LoRA 適配器並合併模型範例 ---")
base_model_for_merge = AutoModelForCausalLM.from_pretrained(
    model_id_lora,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto"
)
# 從保存的路徑載入 LoRA 適配器
lora_model_loaded = PeftModel.from_pretrained(base_model_for_merge, lora_output_path)
# 將 LoRA 適配器與基礎模型權重合併，生成一個可獨立部署的模型
merged_model = lora_model_loaded.merge_and_unload()
print("LoRA 適配器已與基礎模型合併。")
# 儲存合併後的模型
merged_model.save_pretrained("./my_merged_qwen_lora_model")
tokenizer_lora.save_pretrained("./my_merged_qwen_lora_model")
print("合併後的模型已儲存至 ./my_merged_qwen_lora_model，現在可以像普通模型一樣載入和使用。")

### 5.1.4 8-bit 模型載入與 PEFT

結合 8-bit 量化與 LoRA 是在記憶體極其有限的 GPU (例如消費級顯卡，如 8GB 或 12GB) 上訓練大型模型 (例如 7B 或 13B 參數的模型) 的常見策略。

- 8-bit 量化： 在載入模型時，將模型的權重從常見的 float32 或 bfloat16 格式量化為 8-bit 整數。這會大幅減少模型在記憶體中的佔用。

- LoRA： 由於大部分模型參數被量化且凍結，LoRA 僅在少量的可訓練參數上進行更新，這使得訓練過程能夠在有限的記憶體中進行。

> 【注意事項】:量化會導致輕微的精度損失，但對於大多數微調任務，對最終性能的影響通常可以接受。

#### 5.1.4.1 載入 8-bit 量化模型

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 使用一個較小的模型作為 8-bit 示範，例如 EleutherAI/gpt-neo-125m
# 對於更大的模型 (例如 Llama 7B)，這種方法會更有用。
model_id_8bit = "EleutherAI/gpt-neo-125m"
print(f"\n載入 8-bit 量化模型: {model_id_8bit}...")

model_8bit = AutoModelForCausalLM.from_pretrained(
    model_id_8bit,
    load_in_8bit=True, # 【關鍵】啟用 8-bit 量化載入
    device_map="auto", # 自動將模型分佈到可用裝置上
)
tokenizer_8bit = AutoTokenizer.from_pretrained(model_id_8bit)

if tokenizer_8bit.pad_token is None:
    tokenizer_8bit.pad_token = tokenizer_8bit.eos_token
if model_8bit.config.pad_token_id is None:
    model_8bit.config.pad_token_id = model_8bit.config.eos_token_id

print("8-bit 量化模型載入完成！")
print("模型載入後的記憶體佔用會顯著降低。")


載入 8-bit 量化模型: EleutherAI/gpt-neo-125m...


config.json: 0.00B [00:00, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

8-bit 量化模型載入完成！
模型載入後的記憶體佔用會顯著降低。


#### 5.1.4.2 設定 LoRA 配置 (與 8-bit 結合)

In [ ]:
from peft import LoraConfig, TaskType

peft_config_8bit = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none", # 對於 8-bit 量化模型，通常建議將 bias 設為 "none"
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "v_proj"], # 根據模型結構選擇目標模塊
)
print("LoRA 配置 (適用於 8-bit 模型) 完成！")

LoRA 配置 (適用於 8-bit 模型) 完成！


#### 5.1.4.3 設定 SFTConfig (搭配 8-bit + LoRA)

In [ ]:
from trl import SFTConfig, SFTTrainer
from datasets import load_dataset # 再次載入 IMDB 資料集

dataset_8bit = load_dataset("stanfordnlp/imdb", split="train[:3000]")

training_args_8bit = SFTConfig(
    output_dir="./gpt_neo_8bit_lora_sft",
    num_train_epochs=1,
    per_device_train_batch_size=2, # 即使 8-bit 節省記憶體，仍需根據 GPU 實際記憶體調整
    gradient_accumulation_steps=1,
    logging_steps=10,
    save_steps=100,
    learning_rate=2e-4,
    dataset_text_field="text",
    report_to="none"
)

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.


#### 5.1.4.4 初始化 SFTTrainer 與訓練 (8-bit + LoRA)

In [ ]:
print("\n初始化 SFTTrainer (8-bit + LoRA)...")
trainer_8bit = SFTTrainer(
    model=model_8bit, # 傳入 8-bit 量化模型
    train_dataset=dataset_8bit, # 傳入訓練資料集
    args=training_args_8bit,
    peft_config=peft_config_8bit, # 傳入 LoRA 配置
)
print("SFTTrainer (8-bit + LoRA) 初始化完成！")


初始化 SFTTrainer (8-bit + LoRA)...


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/3000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2073 > 2048). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/3000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


SFTTrainer (8-bit + LoRA) 初始化完成！


#### 5.1.4.5【動手試試看】觀察訓練過程。

In [ ]:
print("\n開始訓練 8-bit LoRA 模型 (這將允許您在記憶體有限的 GPU 上訓練更大的模型)。")
trainer_8bit.train()
print("8-bit LoRA 模型訓練完成示範。")
trainer_8bit.save_model("./my_gpt_neo_8bit_lora_model")



開始訓練 8-bit LoRA 模型 (這將允許您在記憶體有限的 GPU 上訓練更大的模型)。


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,3.720100
20,3.814900
30,3.980700
40,3.740200
50,3.963500
60,3.667900
70,3.834500
80,3.545500
90,3.789600
100,3.726500


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during

8-bit LoRA 模型訓練完成示範。


#### 5.1.4.6 上傳適配器到 Hugging Face Hub：
使用 `push_to_hub` 方法。您可以為您的 LoRA 適配器創建一個新的 Repository。

In [ ]:
from huggingface_hub import HfApi
import os

api = HfApi()
repo_id = f"{username}/gpt-neo-125m-lora-imdb" # 例如: "your-username/gpt-neo-125m-lora-imdb"
local_path = "./my_gpt_neo_8bit_lora_model" # 你的 LoRA 適配器儲存路徑

# 創建一個新的模型 Repository
api.create_repo(repo_id=repo_id, exist_ok=True, private=False) # private=True 可以設定為私有

# 上傳適配器檔案
api.upload_folder(
    folder_path=local_path,
    repo_id=repo_id,
    repo_type="model"
)

print(f"LoRA 適配器已成功上傳至 Hugging Face Hub: https://huggingface.co/{repo_id}")

adapter_model.safetensors:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.69k [00:00<?, ?B/s]

LoRA 適配器已成功上傳至 Hugging Face Hub: https://huggingface.co/Heng666/gpt-neo-125m-lora-imdb


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import torch

# 載入基礎模型
base_model_id = "EleutherAI/gpt-neo-125m"
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16, # 注意：推論時通常用 float16 或 bfloat16
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# 載入 LoRA 適配器
lora_model_id = repo_id # "你的HuggingFace用戶名/你的LoRA適配器名稱"
model = PeftModel.from_pretrained(model, lora_model_id)

## 嘗試註解以下看看模型有什麼區別
# 檢查並設定 pad_token_id
# if tokenizer.pad_token is None:
#     # 這是常見的做法：如果沒有pad_token，就用eos_token作為pad_token
#     tokenizer.pad_token = tokenizer.eos_token
#     print(f"tokenizer.pad_token set to eos_token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")

# # 確保模型的config中也有 pad_token_id 的設定
# if model.config.pad_token_id is None:
#     model.config.pad_token_id = tokenizer.pad_token_id
#     print(f"model.config.pad_token_id set to tokenizer.pad_token_id: {model.config.pad_token_id}")

# 推論 (範例)
input_text = "The movie was "
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The movie was  a bit of a disappointment, but it was a good movie. The plot was a bit too much for me, but I was able to enjoy it. The ending was a bit too much for me, but I was able to enjoy it. The




---



# 第 6 章：結論與展望
恭喜您！走到這一步，您已經成功掌握了大型語言模型 (LLM) 監督式微調 (SFT) 的核心概念與實戰技巧。這份 Notebooks 帶您從環境建置開始，逐步深入到 SFTTrainer 的基礎應用、資料格式化、Chat Template 的設定，以及如何利用 LoRA 和 Flash Attention 這樣的關鍵高效能微調策略。

## 6.1 SFT 總結
監督式微調 (SFT) 是將預訓練 LLM 適應特定任務或領域的基石。透過提供明確的輸入-輸出範例，模型能夠學習遵循指令、回答特定問題或生成符合特定風格的文本。

您學會了：
- SFTTrainer 如何簡化複雜的微調流程。
- 如何處理不同格式的資料集，並透過 formatting_prompts_func 進行客製化。
- Chat Template 如何幫助模型理解對話結構和特殊 Token 的重要性。
- LoRA 如何在大幅節省資源的同時，有效提升模型性能。
- 8-bit 量化如何進一步降低記憶體需求，讓您能在資源有限的環境下訓練更大的模型。

這些知識與技能將成為您 LLM 開發道路上的寶貴資產。

## 6.2 其他微調方法簡介
SFT 通常是 LLM 微調的第一步，它讓模型學會基本的行為和知識適應。然而，為了讓模型表現得更像人類，有時還需要進一步的「對齊」和「偏好學習」。雖然我們沒有在這份 Notebooks 中深入實作，但了解以下方法能幫助您規劃未來的學習路徑：

1. 獎勵模型訓練 (Reward Model Training)：
- 目的： 訓練一個模型來評估 LLM 生成的回應品質，判斷回應是「好」還是「壞」。
- 方法： 使用人類對多個 LLM 生成回應的偏好數據 (例如 A 比 B 好) 來訓練一個二元分類器或評分模型
- 作用： 訓練好的獎勵模型會成為後續強化學習階段的「人類偏好模擬器」。

2. 來自人類回饋的強化學習 (Reinforcement Learning from Human Feedback, RLHF)：
- 目的： 讓 LLM 透過與獎勵模型的互動來學習人類偏好，從而生成更高品質、更符合人類期望的回應。
- 方法： 通常結合 SFT、獎勵模型訓練和強化學習 (如 PPO 演算法)。模型會根據獎勵模型的回饋來調整生成策略。
- 作用： 這是讓模型行為更「對齊」人類價值觀的關鍵步驟，許多成功的聊天機器人 (如 ChatGPT) 都使用了類似的技術。

3. **直接偏好優化 (Direct Preference Optimization, DPO)**：
- 目的： 作為 RLHF 的一種更簡潔、更穩定的替代方案，直接利用人類偏好數據來優化模型。
- 方法： DPO 繞過了訓練單獨的獎勵模型和複雜的強化學習訓練過程，直接基於人類偏好數據計算損失函數來更新語言模型。
- 作用： 簡化了對齊過程，且通常能達到與 RLHF 媲美的效果。trl 函式庫中也提供了 DPOTrainer。

這些更進階的微調方法通常在 SFT 之後進行，它們能將模型的性能推向新的高度，使其不僅能產生正確的資訊，還能以更自然、更安全、更符合用戶期望的方式進行互動。